In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

hi, I see this data.
some people analysis method of clustering.
but i see this data which is claasification


In [ ]:
pro=pd.read_csv('/kaggle/input/summer-products-and-sales-in-ecommerce-wish/summer-products-with-rating-and-performance_2020-08.csv')

In [ ]:
print(pro.head(5))
print('''data's information''',pro.info())

first, i chech number of null and delete columns which is 70% percentage of null. because i thint that it's too much to replace

In [ ]:
check=round(pro.isnull().sum()/pro.shape[0],1)
print(check)

In [ ]:
drop_list=[x for x in check.index if check[x] == 1]
pro=pro.drop(drop_list,axis=1)

and number of each column's unique  is 1   
it says to me that it's the same condition.
so it is not imfortant

In [ ]:
column=[]
for i in pro.columns:
    print(i,len(pro[i].unique()))
    if len(pro[i].unique()) ==1:
        column.append(i)

In [ ]:
print(pro[column])

before delete above column, check relation column 
for example, When you buy clothes at an online shopping mall, there are two possible cases: a seasonal sale where you buy clothes for the season or when you buy winter clothes for the summer.
There may be people who buy goods.


In [ ]:
print(pro['theme'].unique())
pro['tags']=pro['tags'].str.contains('summer')
pro=pro.drop(column,axis=1)

second, we distinguish each column' dytype, and 
if categorical column'null replace 'unknown'

In [ ]:
C=(pro.dtypes=='object')
Cate=list(C[C].index)
Integer=(pro.dtypes=='int64')
Float=(pro.dtypes=='float64')
Numer=list(Integer[Integer].index)+list(Float[Float].index)
pro[Cate]=pro[Cate].fillna('unknow')

second_2,
Category data contain the same meaning, which standardizes it.

In [ ]:
def size(x):
    x=x.str.lower().tolist()
    siz=['xxs','xs','s','m','l','xl','xxl','xxxl','xxxxl','xxxxxl','2xl','3xl','4xl','5xl']
    stand_siz=['xxs','xs','s','m','l','xl','xxl','xxxl','xxxxl','xxxxxl']
    for i in range(len(siz)):
        for j in range(len(x)):
            if x[j] == siz[i]:
                x[j]=siz[i]

    
    for i in range(len(x)):
        if x[i] in stand_siz:
            pass
        elif x[i]=='2xl':
            x[i]='xxl'
        elif x[i]=='3xl':
            x[i]='xxxl'
        elif x[i]=='4xl':
            x[i]='xxxxl'
        elif x[i]=='5xl':
            x[i]='xxxxxl'
        else:
            x[i]='other'
    return x

Color_map  = {'NaN':'Unknown','Black':'black','black':'black','White':'white','white':'white','navyblue':'blue',
             'lightblue':'blue','blue':'blue','skyblue':'blue','darkblue':'blue','navy':'blue','winered':'red',
             'red':'red','rosered':'red','rose':'red','orange-red':'red','lightpink':'pink','pink':'pink',
              'armygreen':'green','green':'green','khaki':'green','lightgreen':'green','fluorescentgreen':'green',
             'gray':'grey','grey':'grey','brown':'brown','coffee':'brown','yellow':'yellow','purple':'purple',
             'orange':'orange','beige':'beige'}

In [ ]:
pro['product_variation_size_id']=size(pro['product_variation_size_id'])
pro['product_color']=pro['product_color'].map(Color_map)

In [ ]:
print(pro['product_variation_size_id'].unique())
print(pro['product_color'].unique())

We're going to divide it into categories, but at least if the data in each column exceeds three-thirds, it's meaningless because it's already divided in the index. It also provides unnecessary size for one-hot encoding of data.

In [ ]:
def check_unique(data):
    column=data.columns
    dic={}
    for i in column:
        unique=len(data[i].unique())
        if unique>(data.shape[0])/3:
            dic[i]=unique
            
    return dic

In [ ]:
pro=pro.drop(list(check_unique(pro[Cate])),axis=1)
pro['gap_price']=(pro['price']-pro['retail_price'])/pro['price']
pro=pro.drop(['price','retail_price'],axis=1)

gap_price means between price and retail price percentage.
The reason I did this was to imply the meaning of the two columns by grouping them into one column.

In [ ]:
pro=pro.drop(['rating_five_count','rating_four_count','rating_three_count','rating_two_count','rating_one_count'],axis=1)

The reason for erasing the above column is that information is already in the column called Rating, and the information that comes directly to consumers in selecting the product is the average score and the number of reviews. Some consumers look closely, but due to the nature of the product, there are many types and only information was selected intuitively in the purchase decision.

In [ ]:
C=(pro.dtypes=='object')
Cate=list(C[C].index)
Integer=(pro.dtypes=='int64')
Float=(pro.dtypes=='float64')
Numer=list(Integer[Integer].index)+list(Float[Float].index)

percentage25=pro.describe().loc['25%','units_sold']
percentage50=pro.describe().loc['50%','units_sold']
percentage75=pro.describe().loc['75%','units_sold']
for i in range(pro.shape[0]):
    if pro.iloc[i,0]<percentage25:
        pro.iloc[i,0]=0
    elif (pro.iloc[i,0]>=percentage25) and (pro.iloc[i,0]<=percentage75):
        pro.iloc[i,0]=1
    else:
        pro.iloc[i,0]=2

In [ ]:
print(pro[Numer].describe())

i use box-plot concept. first, i say that this problem is classfication. so The reason why it sold so well is 2 as usual, 1 as usual, and if it doesn't sell abnormally, it's divided by 0.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

The reason why we put the number of members as the unit price of sales is simply to analyze our criteria based on sales volume.

In [ ]:
print(pro.isnull().sum())

In [ ]:
pro['product_color']=pro['product_color'].fillna('unknown')
pro['has_urgency_banner']=pro['has_urgency_banner'].fillna(0)

In [ ]:
numer=pro[Numer]

In [ ]:
print(numer)

Based on earlier predictions from logistic,
We could see quite a few clusters in the rating and the rating count. Then I think we can get meaningful results from a marketing perspective if we focus on the rating and rating count.

In [ ]:
group=[x for x in numer.columns if len(numer[x].unique())<=4]
no_group=[x for x in numer.columns if len(numer[x].unique())>4]

In [ ]:
from scipy.stats import levene
print(pro)

The first assumption, normality, is that if you have more than 30 data, you have normality because of the central theorem.
The second is equal variableness, which is a levene hypothesis test, and if you reject the null hypothesis, use the Welch method.

In [ ]:
gro=pro[group]
rating=pro['rating']
co_rating=pro['rating_count']

In [ ]:
for i in group:
    covar=pro.loc[:,['rating_count',i]]
    unique=pro[i].unique()
    